# Google Maps — Geocoding + Roads (Snap to Roads) — Validador rápido

Valide endereços com **Geocoding API** (detecta `partial_match` e `ZERO_RESULTS`) e ajuste os pontos com **Roads API** (Snap to Roads). Gera mapa Folium e CSV.

## Pré-requisitos
1. **APIs habilitadas**: Geocoding API e Roads API.
2. **Faturamento** ativo no projeto.
3. **API Key** restrita (APIs específicas + HTTP referrer/IP).

## Como usar
- Defina `GOOGLE_API_KEY` como variável de ambiente **ou** edite a célula de config.
- Edite a lista `ADDRESSES`.
- Execute as células em ordem.


In [ ]:
# !pip install requests folium pandas  # use se precisar instalar os pacotes

In [ ]:
import os, math, time, json, requests, pandas as pd
from typing import Optional, Tuple, Dict

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "").strip()
# GOOGLE_API_KEY = "COLE_SUA_CHAVE_AQUI"  # alternativa para testes locais (evite)
if not GOOGLE_API_KEY:
    raise RuntimeError("Defina GOOGLE_API_KEY no ambiente ou edite a célula com a sua chave.")

GEOCODE_URL = "https://maps.googleapis.com/maps/api/geocode/json"
ROADS_URL   = "https://roads.googleapis.com/v1/snapToRoads"

SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "AppMaps-Validator/1.0"})

def _req(url: str, params: Dict, tries: int = 3, backoff: float = 0.7) -> requests.Response:
    last_exc = None
    for i in range(tries):
        try:
            r = SESSION.get(url, params=params, timeout=30)
            if r.status_code == 200:
                return r
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(backoff * (i+1))
                continue
            r.raise_for_status()
            return r
        except Exception as e:
            last_exc = e
            time.sleep(backoff * (i+1))
    if last_exc:
        raise last_exc
    raise RuntimeError("Falha na requisição.")

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlbd = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlbd/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def geocode_address(address: str, region: Optional[str] = "br", language: str = "pt-BR") -> Dict:
    params = {"address": address, "key": GOOGLE_API_KEY, "language": language}
    if region: params["region"] = region
    r = _req(GEOCODE_URL, params); return r.json()

def pick_best_geocode(resp: Dict):
    if not resp or resp.get("status") != "OK": return None
    return resp["results"][0] if resp.get("results") else None

def snap_to_road(lat: float, lon: float, interpolate: bool = False):
    params = {"path": f"{lat},{lon}|{lat},{lon}", "key": GOOGLE_API_KEY, "interpolate": str(interpolate).lower()}
    r = _req(ROADS_URL, params); data = r.json()
    pts = data.get("snappedPoints") or []
    if not pts: return None
    loc = pts[0].get("location", {})
    return (loc.get("latitude"), loc.get("longitude"), pts[0].get("placeId", ""))

In [ ]:
# === Edite os endereços que deseja validar ===
ADDRESSES = [
    "Rua João Pessoa, 100 - Centro, Porto Alegre - RS",
    "Av. Paulista, 1700 - Bela Vista, São Paulo - SP",
    "Rua Inventada Que Não Existe, 123 - São Paulo - SP",
]
ADDRESSES[:3]

In [ ]:
rows = []
for addr in ADDRESSES:
    try:
        g = geocode_address(addr)
        status = g.get("status", "NA")
        best = pick_best_geocode(g)
        if not best:
            rows.append({"input": addr, "status": status, "flag": status})
            continue
        geom = best.get("geometry", {})
        loc = geom.get("location", {})
        lat, lon = loc.get("lat"), loc.get("lng")
        loc_type = geom.get("location_type")
        partial = bool(best.get("partial_match", False))

        snapped = snap_to_road(lat, lon)
        snapped_lat = snapped_lon = place_id = None
        offset = None
        if snapped:
            snapped_lat, snapped_lon, place_id = snapped
            offset = round(haversine_m(lat, lon, snapped_lat, snapped_lon), 1)

        flag = "OK_ROOFTOP" if (not partial and loc_type == "ROOFTOP") else "OK_APROX"
        if partial: flag = "PARTIAL_MATCH"

        rows.append({
            "input": addr, "status": status, "formatted_address": best.get("formatted_address"),
            "partial_match": partial, "location_type": loc_type,
            "lat": lat, "lon": lon,
            "snapped_lat": snapped_lat, "snapped_lon": snapped_lon,
            "snap_place_id": place_id, "snap_offset_m": offset, "flag": flag
        })
    except Exception as e:
        rows.append({"input": addr, "status": "EXCEPTION", "flag": f"EXC:{e}"})

import pandas as pd
df = pd.DataFrame(rows)
df

In [ ]:
import folium, pandas as pd
if not df["lat"].notna().any():
    print("Sem coordenadas para plotar.")
else:
    base_lat = df["lat"].dropna().iloc[0]
    base_lon = df["lon"].dropna().iloc[0]
    m = folium.Map(location=[base_lat, base_lon], zoom_start=12, tiles="OpenStreetMap")
    for _, r in df.dropna(subset=["lat","lon"]).iterrows():
        folium.CircleMarker([r["lat"], r["lon"]], radius=5, fill=True,
                            tooltip=f"Original: {r['input']}\n{r['flag']}").add_to(m)
        if pd.notna(r.get("snapped_lat")) and pd.notna(r.get("snapped_lon")):
            folium.Marker([r["snapped_lat"], r["snapped_lon"]],
                          tooltip=f"Snapped: offset {r.get('snap_offset_m')} m").add_to(m)
            folium.PolyLine([[r["lat"], r["lon"]],[r["snapped_lat"], r["snapped_lon"]]],
                            weight=2, opacity=0.7).add_to(m)
    m

In [ ]:
out_csv = "geocode_roads_validacao.csv"
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
out_csv